# Quaternions 

Quaternions are mathematical objects that extend the concept of complex numbers to 3D space. As for complex numbers, quaternions can both represent points in space and rotations. There a more robust and numerically stable than rotation matrices which makes them powerful for 3D object representations.

A quaternion is defined by 1 real and 3 imaginary numbers.

Theory and formulas from :
- https://en.wikipedia.org/wiki/Quaternions_and_spatial_rotation
- http://x-io.co.uk/res/doc/madgwick_internal_report.pdf#page4

In [2]:
import numpy as np

In [16]:
class quaternion:
    def __init__(self):
        self.coordef(0,0,0,0)
    def coordef(self,r,i,j,k): # defines a quaternion by its 4 coordinates
        self.r=r
        self.i=i
        self.j=j
        self.k=k
    def rotdef(self,thetad,vector): # defines a rotation quaternion : rotation of angle theta around the 3D vector (counter-clk)
        theta=np.deg2rad(thetad)
        vector=vector/np.linalg.norm(vector)
        self.r=np.cos(theta/2)
        self.i=vector[0]*np.sin(theta/2)
        self.j=vector[1]*np.sin(theta/2)
        self.k=vector[2]*np.sin(theta/2)
    def pointdef(self,coordinates): # defines a point quaternion : representation of a point from its 3D cartesian coordinates
        self.r=0
        self.i=coordinates[0]
        self.j=coordinates[1]
        self.k=coordinates[2]
    def show(self): # display a quaternion
        print(self.r,"\t",self.i,"i\t",self.j,"j\t",self.k,"k")

def addqtn(qtn1:quaternion, qtn2:quaternion): #quaternion sum qtn1+qtn2
    qtn=quaternion()
    qtn.r=qtn1.r+qtn2.r
    qtn.i=qtn1.i+qtn2.i
    qtn.j=qtn1.j+qtn2.j
    qtn.k=qtn1.k+qtn2.k
    return qtn

def subqtn(qtn1:quaternion, qtn2:quaternion): # quaternion difference qtn1-qtn2
    qtn=quaternion()
    qtn.r=qtn1.r-qtn2.r
    qtn.i=qtn1.i-qtn2.i
    qtn.j=qtn1.j-qtn2.j
    qtn.k=qtn1.k-qtn2.k
    return qtn

def scaleqtn(alpha, qtn1:quaternion): # scales a quaternion qtn1 by a scalar alpha
    qtn=quaternion()
    qtn.r=alpha*qtn1.r
    qtn.i=alpha*qtn1.i
    qtn.j=alpha*qtn1.j
    qtn.k=alpha*qtn1.k
    return qtn

def multqtn(qtn1:quaternion, qtn2:quaternion): # quaternions multiplication: qtn1 left-multiplies qtn2
    qtn=quaternion()
    qtn.r=qtn1.r*qtn2.r-qtn1.i*qtn2.i-qtn1.j*qtn2.j-qtn1.k*qtn2.k
    qtn.i=qtn1.r*qtn2.i+qtn1.i*qtn2.r+qtn1.j*qtn2.k-qtn1.k*qtn2.j
    qtn.j=qtn1.r*qtn2.j-qtn1.i*qtn2.k+qtn1.j*qtn2.r+qtn1.k*qtn2.i
    qtn.k=qtn1.r*qtn2.k+qtn1.i*qtn2.j-qtn1.j*qtn2.i+qtn1.k*qtn2.r
    return qtn

def conjqtn(qtn1:quaternion): # quaternion conjugate
    qtn=quaternion()
    qtn.r=qtn1.r
    qtn.i=-qtn1.i
    qtn.j=-qtn1.j
    qtn.k=-qtn1.k
    return qtn

def invqtn(qtn1:quaternion): # quaternion multiplication inverse
    qtn=quaternion()
    qtn=scaleqtn(1/modqtn(qtn1)**2,conjqtn(qtn1))
    return qtn

def modqtn(qtn1:quaternion): # quaternion modulus
    return np.linalg.norm([qtn1.r, qtn1.i, qtn1.j, qtn1.k])

def qtn2pt(qtn1:quaternion): # convert back a point quaternion in 3D coordinates
    return [qtn1.i,qtn1.j,qtn1.k]

def qtnrot(qtnr, qtnp): # applies a qtnr rotation on a qtnp point 
    qtn=quaternion()
    qtn=multqtn(qtnr,multqtn(qtnp,invqtn(qtnr)))
    return qtn

def qtn2rotmatrix(q): # display the 3x3 matrix representing a rotation quaternion
    s=1/modqtn(q)**2
    return [1-2*s*(q.j**2+q.k**2),2*s*(q.i*q.j-q.k*q.r),2*s*(q.i*q.k+q.j*q.r)],[2*s*(q.i*q.j+q.k*q.r),1-2*s*(q.i**2+q.k**2),2*s*(q.j*q.k-q.i*q.r)],[2*s*(q.i*q.k-q.j*q.r),2*s*(q.j*q.k+q.i*q.r),1-2*s*(q.i**2+q.j**2)]

def qtn2rot(qtn): # extract the unit vector and angle from a rotation quaternion
    vector=qtn2pt(qtn)
    mod=np.linalg.norm(vector)
    theta=2*np.arctan2(mod,qtn.r)
    return np.rad2deg(theta), vector/mod

def qtn2euler(qtn): #extract the euler angles describing the rotation
    qtn=scaleqtn(1/modqtn(qtn),qtn)    
    psi=np.arctan2(2*(qtn.r*qtn.k+qtn.i*qtn.j),1-2*(qtn.j**2+qtn.k**2))
    theta=np.arcsin(2*(qtn.r*qtn.j-qtn.k*qtn.i))
    phi=np.arctan2(2*(qtn.r*qtn.i+qtn.j*qtn.k),1-2*(qtn.i**2+qtn.j**2))
    return np.rad2deg(psi),np.rad2deg(theta),np.rad2deg(phi)
    
if __name__ == "__main__":
    print("Define a general quaternion from its coordinates, e.g. a=1 + 2i + 3j + 4k")
    a=quaternion()
    a.coordef(1, 2, 3, 4)
    a.show()
    print("Define a rotation quaternion (angle=90°, vector=[1,1,1])")
    b=quaternion()
    b.rotdef(90,[1,1,1])
    b.show()
    print("Add the two previous quaternions")
    c=addqtn(a,b)
    c.show()
    print("Conjuguate the previous quaternions")
    c=conjqtn(c)
    c.show()
    print("Compute its modulus")
    print(modqtn(c))
    print("")
    angle=120
    vector=[2,0,0]
    point=[1,0,0]
    print("Quaternion associated to a ",angle, "° rotation along ", vector," is:")
    rotation=quaternion()
    rotation.rotdef(angle,vector)
    rotation.show()
    print("Quaternion associated to point (",point,") is:")
    pointq=quaternion()
    pointq.pointdef(point)
    pointq.show()
    print("Apply rotation on point")
    out=qtn2pt(qtnrot(rotation,pointq))
    print("Result=", out)
    print("Equivalent rotation matrix")
    print(qtn2rotmatrix(rotation))
    print("Rotation parameters extracted from quaternion")
    print(qtn2rot(rotation))
    print("Equivalent euler angles extracted from quaternion [psi,theta,phi]")
    print(qtn2euler(rotation))

Define a general quaternion from its coordinates, e.g. a=1 + 2i + 3j + 4k
1 	 2 i	 3 j	 4 k
Define a rotation quaternion (angle=90°, vector=[1,1,1])
0.7071067811865476 	 0.4082482904638631 i	 0.4082482904638631 j	 0.4082482904638631 k
Add the two previous quaternions
1.7071067811865475 	 2.408248290463863 i	 3.408248290463863 j	 4.408248290463863 k
Conjuguate the previous quaternions
1.7071067811865475 	 -2.408248290463863 i	 -3.408248290463863 j	 -4.408248290463863 k
Compute its modulus
6.305765836971956

Quaternion associated to a  120 ° rotation along  [2, 0, 0]  is:
0.5000000000000001 	 0.8660254037844386 i	 0.0 j	 0.0 k
Quaternion associated to point ( [1, 0, 0] ) is:
0 	 1 i	 0 j	 0 k
Apply rotation on point
Result= [1.0, 0.0, 0.0]
Equivalent rotation matrix
([1.0, 0.0, 0.0], [0.0, -0.4999999999999998, -0.8660254037844388], [0.0, 0.8660254037844388, -0.4999999999999998])
Rotation parameters extracted from quaternion
(119.99999999999999, array([1., 0., 0.]))
Equivalent euler angle